In [1]:
import pandas as pd
import os
import win32com.client
import os
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import zipfile
import time

#簡單來說即是暴力展開另存新檔  file沒加xls filename有加

In [2]:
class base:
    def __init__(self,WhereExcel,PlaceFile,Filename):
        self.WhereExcel=WhereExcel
        self.PlaceFile=PlaceFile
        self.Filename=Filename

In [3]:
class file():
    def  createPuttxtdir(self,path):
        '''創建要放txt的file'''
        if  not os.path.isdir(path):
            os.mkdir(path)    
    def  excel_recover(self,path_where,path_put):
        #path_where損毀資料放的地方,path_put要放資料的地方
        self.createPuttxtdir(path_put)
        if os.path.isdir(path_put):
            a=os.listdir(path_put)
        for filename in os.listdir(path_where):
            if not( (filename.split('.')[0]+'.xlsx') in a):
                file= os.path.splitext(filename)[0]
                o = win32com.client.Dispatch("Excel.Application")
                o.Visible = False
                wb = o.Workbooks.Open(path_where+'\\' + filename) 
                wb.ActiveSheet.SaveAs(path_put+'\\' + file + ".xlsx", 51) 
                o.Application.Quit()
    def   read_excel_combine_chinese(self,path_where):
        #path_where處理好的excel放哪裡
        list_combine=[]
        a=pd.read_excel(path_where,usecols=["英文名稱", "中文名稱"])
        c,d=a.shape
        for i in range (c):
            list_combine=list_combine+[[a.iloc[i]['英文名稱'],a.iloc[i]['中文名稱']]]
        return list_combine
    def  store_txt(self,path_where,path_put):
        self.createPuttxtdir(path_put)
        a=os.listdir(path_where)
        for i in a:
            b=self.read_excel_combine_chinese(path_where+'\\'+i)
            txt=open(path_put+'\\'+i.strip('.xlsx')+'.txt','w',encoding='utf-8')
            for k in b:
                for n in k:
                    txt.write(str(n))
                txt.write('\n')
    def  total_resolve(self,broke_excelpath,put_file_large):
        #輸入損毀的excel path,輸入要放置的path
        self.createPuttxtdir(put_file_large)
        self.createPuttxtdir(put_file_large+'\\'+'correct')
        self.excel_recover(broke_excelpath,put_file_large+'\\correct')
        self.store_txt(put_file_large+'\\correct',put_file_large+'\\'+'txtsolve')

之後修改背景執行

In [57]:
class download():
    def getinfo_name(self):
        #得到所有資料
        k=[]
        response = requests.get("https://terms.naer.edu.tw/download/")
        soup = BeautifulSoup(response.text, "html.parser")
        for i in soup.find_all('li'):
            if '最後修改' in str(i):
                i=str(i).strip('<li>')
                i=i.split('<')
                i=i[1]
                i=i.split('>')
                i=i[1]
                k=k+[i]
        return k
    def new_report(self,test_report):
        #輸入文件夾並會得到最新的文件名稱
        lists = os.listdir(test_report)                                    #列出目錄的下所有文件和文件夾保存到lists
        #print(list)
        lists.sort(key=lambda fn:os.path.getmtime(test_report + "\\" + fn))#按時間排序
        file_new = os.path.join(test_report,lists[-1])                     #獲取最新的文件保存到file_new
        #print(file_new)
        return file_new
    def download_all(self,filenamelist,putdownload_path):
        #上方名稱表單 放哪裡
        domain_url = 'https://terms.naer.edu.tw/download/'
        chrome_options = webdriver.ChromeOptions()
        prefs={'profile.default_content_settings.popus':0,
              'download.default_directory':putdownload_path,
              'profile.default_content_setting_values.automatic_downloads':1}
        chrome_options.add_experimental_option('prefs',prefs)
        chrome_options.add_experimental_option('excludeSwitches',['enable-automation'])
        browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
        browser.get(domain_url)
        for i in filenamelist:
            try:
                a=browser.find_element_by_partial_link_text(i)
                a.click()
                time.sleep(1)
                b=browser.find_element_by_partial_link_text(i+'壓縮檔')
                b.click()
                time.sleep(1)
                browser.back()
                p=self.new_report(putdownload_path)
                os.rename(p,putdownload_path+'\\'+i+'.rar')
            except:
                continue
        time.sleep(1)
        browser.close()
    def unzipfile(self,where_path,put_path):
        path=where_path
        j=os.listdir(put_path)
        for t in range(len(j)):
            j[t]=j[t].split('.')[0][0:-1]
        for i in os.listdir(where_path):
            p=i.split('.')[0]
            if not(p in j):
                zip_file = zipfile.ZipFile(path+'\\'+i)
                zip_list = zip_file.namelist() # 得到压缩包里所有文件
                count=0
                for f in zip_list: 
                    zip_file.extract(f, put_path) # 循环解压文件到指定目录
                    p=self.new_report(put_path)
                    os.rename(p,put_path+'\\'+i.strip('.rar')+str(count)+'.'+p.split('.')[1])
                    count=count+1
        zip_file.close()

In [58]:
m=file()
n=download()

下載資料

In [54]:
p=n.getinfo_name()
n.download_all(p,'F:\\Downloads')
m.unzipfile('F:\\Downloads','F:\\doctor')

C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_6376/208975642.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_6376/208975642.py:33: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_6376/208975642.py:37: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  a=browser.find_element_by_partial_link_text(i)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_6376/208975642.py:40: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  b=browser.find_element_by_partial_link_text(i+'壓縮檔')


AttributeError: 'file' object has no attribute 'unzipfile'

解壓縮資料

In [59]:
n.unzipfile('F:\\Downloads','F:\\doctor')

修復資料

In [60]:
m.total_resolve('F:\\doctor','F:\\correct')

保存區

['Term_400_', '人體解剖學', '力學名詞', '土木工程名詞', '土木工程名詞', '土木工程名詞']

In [ ]:
  
domain_url = 'https://terms.naer.edu.tw/download/'
chrome_options = webdriver.ChromeOptions()
prefs={'profile.default_content_settings.popus':0,
      'download.default_directory':'F:\\Downloads',
      'profile.default_content_setting_values.automatic_downloads':1}
chrome_options.add_experimental_option('prefs',prefs)
chrome_options.add_experimental_option('excludeSwitches',['enable-automation'])
browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
browser.get(domain_url)
for i in k:
    try:
        a=browser.find_element_by_partial_link_text(i)
        a.click()
        time.sleep(1)
        b=browser.find_element_by_partial_link_text(i+'壓縮檔')
        b.click()
        time.sleep(1)
        browser.back()
    except:
        continue
time.sleep(1)
browser.close()

C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_12088/908646745.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_12088/908646745.py:8: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('F:\\chromedriver',chrome_options=chrome_options)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_12088/908646745.py:12: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  a=browser.find_element_by_partial_link_text(i)
C:\Users\GIGABYTE\AppData\Local\Temp/ipykernel_12088/908646745.py:15: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  b=browser.find_element_by_partial_link_text(i+'壓縮檔')
Exception ignored in: <func

In [ ]:
a=pd.read_excel("F:\\correct\\Term_374_0.xlsx",usecols=["英文名稱", "中文名稱"])
c,d=a.shape
for i in range (c):
    print(a.iloc[i]['中文名稱'])

In [ ]:
import win32com.client
import os

file_dir = "F:\\doctor" 
for filename in os.listdir(file_dir):
    print(filename)
    file= os.path.splitext(filename)[0]
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False
    wb = o.Workbooks.Open("F:\\doctor\\" + filename) 
    wb.ActiveSheet.SaveAs("F:\\correct\\" + file + ".xlsx", 51) 
    o.Application.Quit()

In [ ]:
import zipfile

path='F:\\Downloads'
j=os.listdir('F:\\doctor')
for t in range(len(j)):
    j[t]=j[t].split('.')[0][0:-1]
for i in os.listdir('F:\\Downloads'):
    p=i.split('.')[0]
    if not(p in j):
        zip_file = zipfile.ZipFile(path+'\\'+i)
        zip_list = zip_file.namelist() # 得到压缩包里所有文件
        count=0
        for f in zip_list: 
            zip_file.extract(f, 'F:\\doctor') # 循环解压文件到指定目录
            p=m.new_report('F:\\doctor')
            os.rename(p,'F:\\doctor'+'\\'+i.strip('.rar')+str(count)+'.'+p.split('.')[1])
            count=count+1
zip_file.close()
